In [ ]:
# install necessary geospatial packages

!pip -q install rasterio
!pip -q install fiona
!pip -q install geopandas
!pip -q install pycrs
!pip -q install descartes 
!pip -q install pypng
!pip -q install rtree
!pip -q install pygeos

#Built in breakpoint function Not sure how it works yet
import pdb

     |████████████████████████████████| 19.3 MB 1.5 MB/s 
     |████████████████████████████████| 16.7 MB 193 kB/s 
     |████████████████████████████████| 1.0 MB 29.0 MB/s 
     |████████████████████████████████| 6.3 MB 61.2 MB/s 
     |████████████████████████████████| 48 kB 4.4 MB/s 
     |████████████████████████████████| 994 kB 29.3 MB/s 
     |████████████████████████████████| 2.1 MB 23.0 MB/s 


In [ ]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

 # necessary basic libraries
import pandas as pd
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import json
import png
import re
import time
import glob

# geospatial libraries
import rasterio
import geopandas
from geopandas.tools import sjoin
import fiona
from rasterio.plot import show
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs
import descartes
import pygeos
import rtree

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
# just defining a function we are going to use shortly.

def getFeatures(gdf):
        """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
        return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
# Filepath to tiff that you want to read
tiff_filepath = '/content/drive/MyDrive/forestseg/paracou_data/imagery/RGB_Broad_Mosaics/Paracou_20220106_RGB_5cm_mosa_rect.tif'
# Filepath to Read in shapefile of crowns, if training on your own data!
manuallabels_filepath = '/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/220303_AllSpLabelled.gpkg'

#Filepath to csv containing the origin x and y coordinates of the tiles you wish to slice up...
csvfile_filepath = '/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/training_tiles_paracu_broad_mosaics_50.csv'
#
save_tiff_filepath = '/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/ForTrainingSegmentation'

#
save_png_filepath = '/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/ForTrainingSegmentation'
save_geojson_filepath = '/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/ForTrainingSegmentation'

In [ ]:
directory = '/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/ForTrainingTreeSpecies/tofindtreespecies'
List_GenusSpecies = []
for filename in [file for file in os.listdir(directory) if file.endswith('.geojson')]:
    json_file = os.path.join(directory, filename)
    with open(json_file) as f:
        img_anns = json.load(f)
        #filename = os.path.join(directory, img_anns["imagePath"])

        for features in img_anns['features']:
          anno = features['geometry']
        #pdb.set_trace()
          GenusSpecies = features['properties']['Genus_Species']
          
          List_GenusSpecies.append(GenusSpecies)

#print(List_GenusSpecies)
GenusSpecies_WithReps = List_GenusSpecies
from collections import Counter
c = Counter(GenusSpecies_WithReps)


In [ ]:
#   os.mkdir(path)
#   os.makedirs(path)  
#newpath = r'C:\Program Files\arbitrary' 
#if not os.path.exists(newpath):
#   os.makedirs(newpath)
# example code how its done
# All files and directories ending with .txt and that don't begin with a dot:
#print(glob.glob("/home/pc/*.txt")) 
# All files and directories ending with .txt with depth of 2 folders, ignoring names beginning with a dot:
#print(glob.glob("/home/pc/*/*.txt")) 

#os.path.abspath("mydir/myfile.txt")

#Filepath with all the imagery
imagery_filepath = '/content/drive/MyDrive/forestseg/paracou_data/imagery/RGB_Broad_Mosaics'
list_tifs = glob.glob(imagery_filepath + "/*.tif")
for k in  range(1,len(list_tifs)):

  # Read in shapefile of crowns, if training on your own data!
  crowns = geopandas.read_file(manuallabels_filepath)

  # have a look at the crowns if we like
  crowns.head(3)

  print("imagery ", k)
  tiff_filepath = list_tifs[k]

  FilePathSplit = tiff_filepath.split("/")
  FileName = FilePathSplit[-1]

  #Also need this without the '.tif'
  FileNameSave = FileName.split(".tif")
  FileNameSave = FileNameSave[0]
  #print(FileNameSave)

  #Extract Resolution Information from imagery
  Resolution1 = re.search('(?!RGB_)\d+(?=cm_)', FileName)
  Resolution1 = Resolution1.group(0)
  #Extract Date from imagery
  Date = re.search('([0-9])\d+', FileName)
  Date = Date.group(0)

  #date1 = "31/12/2015"
  #You can do the following:
  #newdate1 = time.strptime(date1, "%d/%m/%Y")

  DateImage = time.strptime(Date, "%Y%m%d")
  #Make comparisons
  print(DateImage)
  sizecrowns = np.size(crowns.StartDate)
  for i in reversed(range(sizecrowns)):
    if crowns.StartDate[i] == None:
      continue
    else:
      print("StartDate ",i," ",crowns.StartDate[i])
      TreeStartDate = time.strptime(crowns.StartDate[i], "%Y-%m-%d")
      if DateImage >= TreeStartDate:
        continue
      else:
        crowns.drop(i, inplace=True)
        print('dropped ',i)

# Remove trees with endate before the imagery wat taken
  for i in reversed(range(sizecrowns)):
    if i in crowns.index:
      if crowns.EndDate[i] == None:
        continue
      else:
        print("EndDate", i," ",crowns.EndDate[i])
        TreeEndDate = time.strptime(crowns.EndDate[i], "%Y-%m-%d")
        if DateImage <= TreeEndDate:
          continue
        else:
          crowns.drop(i, inplace=True)
          print('dropped ',i)
    else:
      continue
# Remove Indet trees
   for i in reversed(range(sizecrowns)):
     if i in crowns.index:
       if "Indet" in crowns.Genus_Species[i]:  
         print(i," ", crowns.Genus_Species[i])
         crowns.drop(i, inplace=True)
     else:
       continue

# Remove species of trees that we have less than 3 in our training set
   for i in reversed(range(sizecrowns)):
     if i in crowns.index:
       if c[crowns.Genus_Species[i]] < 3:
           print(i," ", crowns.Genus_Species[i])
           crowns.drop(i, inplace=True)
       else:
         continue
     else:
       continue
  crowns_polygon = crowns.explode('geometry',index=False)
  crowns_polygon.head(3)

  
  newpath = save_png_filepath # + '/' + FileNameSave
  #if not os.path.exists(newpath):
  #  os.makedirs(newpath)
  if not os.path.exists(newpath + '/test'):
    os.makedirs(newpath + '/test')

  if not os.path.exists(newpath + '/train'):
    os.makedirs(newpath + '/train')

  if not os.path.exists(newpath + '/' + FileNameSave):
    os.makedirs(newpath + '/' + FileNameSave)
    
  crowns_polygon.to_file(newpath + '/' + FileNameSave + '/LabelData.gpkg', layer='countries', driver="GPKG")

  # Read in a tiff file, and the csv of the tiles we expect
  data = rasterio.open(tiff_filepath)

  # Read in the shapefile of manual crowns - again, this is only if you are wanting to train with your own crowns

  #crowns = geopandas.read_file(manuallabels_filepath)

  # read in a csv containing the origin x and y coordinates of the tiles you wish to slice up...
  # this file can be commented out and replaced by looping over x and y coordinates as below

  tiles = pd.read_csv(csvfile_filepath)

  # set the desired buffer, tile width, heght and resolution of the tiff tiles. Suggested values are given below. 
  # The buffer will be affected by the area of tree crowns in your region of forest.

  buffer = 20
  tile_width = 50
  tile_height = 50
  #
  #resolution = float(Resolution1) / 100. # in metres per pixel
  #scaling = 1/resolution  # scaling parameter to transform the shapefile coordinates so they match the png


  # the for loop can be rewritten to work with out requiring a csv e.g. for minx in range(V, W, step): e.g.:
  #for minx in range(286900,286950,tile_width):
  # for miny in range(584000,584050,tile_height):
    
  list_test_names= ['tile_286400_583662','tile_286450_583662','tile_286748_583880','tile_286798_583880','tile_286952_584044','tile_287002_583994','tile_287295_584130']   

  for ind in tiles.index:
      minx = tiles['Origin X coordinate'][ind] 
      miny = tiles['Origin Y coordinate'][ind]
      print(minx,miny)
      RejectImage = False
      # define the bounding box of the whole tile, including the buffer
      bbox = box(minx-buffer, miny-buffer, minx+tile_width+buffer, miny+tile_height+buffer)
      # define the bounding box of the tile, excluding the buffer (hence selecting just the central part of the tile)
      bbox_central = box(minx, miny, minx+tile_width, miny+tile_height)
      # turn the bounding boxes into geopandas DataFrames, need to set the crs
      geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(32622))
      geo_central = gpd.GeoDataFrame({'geometry': bbox_central}, index=[0], crs=from_epsg(32622))
      #print(geo_central)

      ### here we are cropping the tiff to the bounding box of the tile we want
      coords = getFeatures(geo)
      print(coords)
      
      # define the tile as a mask of the whole tiff with just the bounding box
      
      out_img, out_transform = mask(data, shapes=coords, crop=True)

      # copy the metadata
      out_meta = data.meta.copy()
      #print(out_meta)
      epsg_code = int(data.crs.data['init'][5:])
      #print(epsg_code)
      
      # update the metadata
      out_meta.update({"driver": "GTiff",
                      "height": out_img.shape[1],
                      "width": out_img.shape[2],
                      "transform": out_transform
                      })
      
      # here we are saving the tile as a new tiff, named by the origin of the tile
      newpath_tiff = save_tiff_filepath + '/' + FileNameSave +'/tiffs' 
      if not os.path.exists(newpath_tiff):
        os.makedirs(newpath_tiff)
      out_tif = newpath_tiff + '/tile_' + str(minx) + '_' + str(miny) + '.tif'
      with rasterio.open(out_tif, "w", **out_meta) as dest:
                        dest.write(out_img)
      #plt.imshow(out_tif)
      # read in the tile we have just saved
      clipped = rasterio.open(newpath_tiff + '/tile_' + str(minx) + '_' + str(miny) + '.tif')
      # read it as an array
      arr = clipped.read()
      
      # check the shape of the tile if you wish
      print('Array Shape')
      print(arr.shape)

      arr.shape[1]
      resolution =  (2 * buffer + tile_width) / arr.shape[1]           #float(Resolution1) / 100. in metres per pixel
      scaling = 1/resolution  # scaling parameter to transform the shapefile coordinates so they match the png
      print("Resolution = ", resolution, "Scaling = ", scaling)
      #pdb.set_trace()

      # each band of the tiled tiff is a colour!
      R = arr[0]
      G = arr[1]
      B = arr[2]
      
      # stack up the bands in an order appropriate for saving with cv2, then rescale to the correct 0-255 range for cv2
      
      rgb = np.dstack((B,G,R)) # BGR for cv2
      #rgb_rescaled = 255*rgb/65535 # scale the values of the bands if they are non-standard to range 0-255
      rgb_rescaled = rgb # usually rescaling is not required, but it depends on your tiff
      
      # Remove images that have less than 50% valid regions
      rgb_average = (rgb_rescaled[:,:,0].astype('uint16') + rgb_rescaled[:,:,1].astype('uint16')  + rgb_rescaled[:,:,2].astype('uint16') ) / 3.
      rgb_mask = np.where(rgb_average > 250, 1 , 0)
      NumerOfPixels = rgb_mask.sum()
      TotalPixels = rgb_mask.shape[0] * rgb_mask.shape[1]
      if NumerOfPixels > 0.5 * TotalPixels:
        RejectImage = True
      
      if RejectImage == False:
        

        # save this as jpg or png...we are going for png...again, named with the origin of the specific tile
        
        
        ### now we have dealt with tiling the tiff, we want to deal with tiling the crowns...
        ### IF we have manual crowns we are going to use for training
        ### THEN uncomment the following 4 lines of code
        
        ### select the crowns that intersect the non-buffered central section of the tile using the inner join
        overlapping_crowns = sjoin(crowns_polygon, geo_central, how="inner")
        
        #print(overlapping_crowns)
        ### translate to 0,0 to overlay on png
        moved = overlapping_crowns.translate(-minx+buffer, -miny+buffer)  
      
        ### scale to deal with the resolution
        moved_scaled = moved.scale(scaling, scaling, origin=(0, 0))

        Geodf_moved_scaled = gpd.GeoDataFrame(geometry=gpd.GeoSeries(moved_scaled))
        overlapping_crowns.geometry = Geodf_moved_scaled.geometry

        #pdb.set_trace()
        # { imagePath = [...]} to be added to json
        if moved_scaled.empty == False:
        ### save as a geojson, a format compatible with detectron2, again named by the origin of the tile
          #overlapping_crowns.plot()
          Name_Tile= 'tile_'+str(minx)+'_'+str(miny)
          inside = 0;
          for l in range(len(list_test_names)):
            if list_test_names[l] in Name_Tile:
                inside = inside + 1
          if inside == 1:
            cv2.imwrite(newpath + '/test/' + FileNameSave + '_tile_'+str(minx)+'_'+str(miny)+'.png', rgb_rescaled.astype('uint8'))
            overlapping_crowns.to_file(driver = 'GeoJSON', filename= newpath + '/test/' + FileNameSave + '_tile_'+str(minx)+'_'+str(miny)+'.geojson')
          else:
            cv2.imwrite(newpath + '/train/' + FileNameSave + '_tile_'+str(minx)+'_'+str(miny)+'.png', rgb_rescaled.astype('uint8'))
            overlapping_crowns.to_file(driver = 'GeoJSON', filename= newpath + '/train/' + FileNameSave + '_tile_'+str(minx)+'_'+str(miny)+'.geojson')
        else:
          print('Empty of Training Data tile_'+str(minx)+'_'+str(miny))

imagery  1
time.struct_time(tm_year=2020, tm_mon=11, tm_mday=5, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=310, tm_isdst=-1)
StartDate  3119   2020-10-23
StartDate  3116   2020-10-23
StartDate  3053   2021-03-03
dropped  3053
StartDate  3039   2021-03-03
dropped  3039
StartDate  2946   2020-10-23
StartDate  2681   2019-11-15
StartDate  2674   2019-11-15
StartDate  2668   2019-11-14
StartDate  2665   2019-11-15
StartDate  2664   2019-11-15
StartDate  2656   2019-11-15
StartDate  2627   2019-11-15
StartDate  2520   2019-11-15
StartDate  2515   2019-11-15
StartDate  2346   2019-11-15
StartDate  2345   2019-11-15
StartDate  1701   2019-11-15
EndDate 3120   2021-03-03
EndDate 3118   2020-10-23
dropped  3118
EndDate 3117   2020-10-23
dropped  3117
EndDate 2839   2020-10-23
dropped  2839
EndDate 2714   2021-08-02
EndDate 2669   2019-11-15
dropped  2669
EndDate 2614   2020-01-23
dropped  2614
EndDate 2549   2021-03-03
EndDate 2231   2019-11-15
dropped  2231
EndDate 1895   2019-11-15
dro

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.


286400 583762
[{'type': 'Polygon', 'coordinates': [[[286470.0, 583742.0], [286470.0, 583832.0], [286380.0, 583832.0], [286380.0, 583742.0], [286470.0, 583742.0]]]}]


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Array Shape
(3, 2276, 2276)
Resolution =  0.03954305799648506 Scaling =  25.288888888888888
286400 583662
[{'type': 'Polygon', 'coordinates': [[[286470.0, 583642.0], [286470.0, 583732.0], [286380.0, 583732.0], [286380.0, 583642.0], [286470.0, 583642.0]]]}]


...

Array Shape
(3, 2524, 2524)
Resolution =  0.03565768621236133 Scaling =  28.044444444444444
287226 584207
[{'type': 'Polygon', 'coordinates': [[[287296.0, 584187.0], [287296.0, 584277.0], [287206.0, 584277.0], [287206.0, 584187.0], [287296.0, 584187.0]]]}]


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Array Shape
(3, 2524, 2525)
Resolution =  0.03565768621236133 Scaling =  28.044444444444444
Empty of Training Data tile_287226_584207
286352 583702


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


[{'type': 'Polygon', 'coordinates': [[[286422.0, 583682.0], [286422.0, 583772.0], [286332.0, 583772.0], [286332.0, 583682.0], [286422.0, 583682.0]]]}]
Array Shape
(3, 2524, 2524)
Resolution =  0.03565768621236133 Scaling =  28.044444444444444


In [ ]:
list_test_names= ['tile_286400_583662','tile_286450_583662','tile_286748_583880','tile_286798_583880','tile_286952_584044','tile_287002_583994','tile_287295_584130']


In [ ]:
minx = 286450
miny = 583662
testaa =  'random_path/with/whisebfgoua' + '/tile_'+str(minx)+'_'+str(miny)+'.geojson'
inside = 0;
for l in range(len(list_test_names)-1):
  if list_test_names[l] in testaa:
    inside = inside + 1
if inside == 1:
  #printhere
else:
  #printhere

In [ ]:
arr

      
#Just need to check the above are correct

In [ ]:
directory = '/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/ForTrainingTreeSpecies/train'
List_GenusSpecies = []
for filename in [file for file in os.listdir(directory) if file.endswith('.geojson')]:
    json_file = os.path.join(directory, filename)
    with open(json_file) as f:
        img_anns = json.load(f)
        #filename = os.path.join(directory, img_anns["imagePath"])

        for features in img_anns['features']:
          anno = features['geometry']
        #pdb.set_trace()
          GenusSpecies = features['properties']['Genus_Species']
          
          List_GenusSpecies.append(GenusSpecies)

#print(List_GenusSpecies)
WithReps = List_GenusSpecies
UniqueList1 = list(set(List_GenusSpecies))
print(UniqueList1)

['Oenocarpus_bataua', 'Chrysophyllum_prieurii', 'Parinari_campestris', 'Licaria_martiniana', 'Hymenopus_heteromorphus', 'Chaetocarpus_schomburgkianus', 'Bagassa_guianensis', 'Eschweilera_coriacea', 'Tapirira_guianensis', 'Lecythis_poiteaui', 'Protium_opacum', 'Eschweilera_sagotiana', 'Dicorynia_guianensis', 'Parkia_nitida', 'Ampelocera_edentula', 'Pourouma_guianensis', 'Eperua_falcata', 'Recordoxylon_speciosum', 'Tabebuia_insignis', 'Cecropia_obtusa', 'Inga_loubryana', 'Platonia_insignis', 'Pourouma_melinonii', 'Couratari_multiflora', 'Brosimum_rubescens', 'Sterculia_pruriens', 'Qualea_rosea', 'Pouteria_ambelaniifolia', 'Licania_micrantha', 'Vouacapoua_americana', 'Virola_surinamensis', 'Licania_ovalifolia', 'Aspidosperma_album', 'Couma_guianensis', 'Symphonia_sp.1', 'Chrysophyllum_sanguinolentum', 'Licania_membranacea', 'Ormosia_coutinhoi', 'Anacardium_spruceanum', 'Inga_pezizifera', 'Pradosia_cochlearia', 'Chrysophyllum_pomiferum', 'Pouteria_engleri', 'Albizia_pedicellaris', 'Parinar

In [ ]:
df = pd.DataFrame (WithReps, columns = ['GenusSpecies'])
from collections import Counter

c = Counter( WithReps )
print(c)
c['Eperua_falcata']
type(c)
c['Pradosia_cochlearia'] > 2

Counter({'Pradosia_cochlearia': 23, 'Eperua_falcata': 19, 'Dicorynia_guianensis': 14, 'Eschweilera_sagotiana': 13, 'Eperua_grandiflora': 12, 'Symphonia_sp.1': 9, 'Sextonia_rubra': 8, 'Vouacapoua_americana': 7, 'Sterculia_pruriens': 7, 'Tapura_capitulifera': 6, 'Pouteria_eugeniifolia': 6, 'Recordoxylon_speciosum': 6, 'Chrysophyllum_prieurii': 5, 'Platonia_insignis': 5, 'Chrysophyllum_pomiferum': 3, 'Parkia_nitida': 3, 'Goupia_glabra': 3, 'Carapa_surinamensis': 3, 'Licania_alba': 3, 'Bocoa_prouacensis': 3, 'Lueheopsis_rugosa': 3, 'Parinari_montana': 2, 'Anacardium_spruceanum': 2, 'Bagassa_guianensis': 2, 'Parkia_pendula': 2, 'Eschweilera_coriacea': 2, 'Protium_opacum': 2, 'Parinari_campestris': 2, 'Virola_surinamensis': 2, 'Oenocarpus_bataua': 2, 'Tapirira_guianensis': 2, 'Qualea_rosea': 2, 'Pouteria_ambelaniifolia': 1, 'Didymopanax_decaphyllus': 1, 'Pouteria_engleri': 1, 'Licania_ovalifolia': 1, 'Pourouma_melinonii': 1, 'Humiriastrum_subcrenatum': 1, 'Brosimum_rubescens': 1, 'Licania_me

collections.Counter

In [ ]:
#c['Pradosia_cochlearia'] > 2
WithReps

['Pouteria_ambelaniifolia',
 'Tapura_capitulifera',
 'Eperua_grandiflora',
 'Chrysophyllum_pomiferum',
 'Didymopanax_decaphyllus',
 'Parinari_montana',
 'Eperua_grandiflora',
 'Tapura_capitulifera',
 'Eperua_grandiflora',
 'Eschweilera_sagotiana',
 'Parkia_nitida',
 'Dicorynia_guianensis',
 'Pradosia_cochlearia',
 'Pradosia_cochlearia',
 'Anacardium_spruceanum',
 'Chrysophyllum_pomiferum',
 'Pouteria_engleri',
 'Chrysophyllum_prieurii',
 'Sextonia_rubra',
 'Eperua_grandiflora',
 'Pradosia_cochlearia',
 'Eperua_falcata',
 'Eschweilera_sagotiana',
 'Sextonia_rubra',
 'Licania_ovalifolia',
 'Goupia_glabra',
 'Symphonia_sp.1',
 'Sextonia_rubra',
 'Eperua_falcata',
 'Pouteria_eugeniifolia',
 'Pourouma_melinonii',
 'Sextonia_rubra',
 'Vouacapoua_americana',
 'Eperua_falcata',
 'Anacardium_spruceanum',
 'Vouacapoua_americana',
 'Vouacapoua_americana',
 'Dicorynia_guianensis',
 'Eperua_falcata',
 'Dicorynia_guianensis',
 'Pradosia_cochlearia',
 'Pradosia_cochlearia',
 'Pradosia_cochlearia',
 '

In [ ]:
directory = '/content/drive/MyDrive/forestseg/paracou_data/Panayiotis_Outputs/ForTrainingTreeSpecies/test'
List_GenusSpecies = []
for filename in [file for file in os.listdir(directory) if file.endswith('.geojson')]:
    json_file = os.path.join(directory, filename)
    with open(json_file) as f:
        img_anns = json.load(f)
        #filename = os.path.join(directory, img_anns["imagePath"])

        for features in img_anns['features']:
          anno = features['geometry']
        #pdb.set_trace()
          GenusSpecies = features['properties']['Genus_Species']
          
          List_GenusSpecies.append(GenusSpecies)

#print(List_GenusSpecies)
UniqueList = list(set(List_GenusSpecies))
print(UniqueList)

['Pradosia_cochlearia', 'Eschweilera_congestiflora', 'Manilkara_huberi', 'Terminalia_grandis', 'Tapura_capitulifera', 'Sloanea_granulosa', 'Symphonia_globulifera', 'Guatteria_schomburgkiana', 'Eperua_grandiflora', 'Sacoglottis_guianensis', 'Carapa_surinamensis', 'Tachigali_melinonii', 'Parkia_nitida', 'Vouacapoua_americana', 'Licania_ovalifolia', 'Manilkara_bidentata', 'Dicorynia_guianensis', 'Protium_opacum', 'Dendrobangia_boliviana', 'Sextonia_rubra', 'Protium_stevensonii', 'Licania_micrantha', 'Platonia_insignis', 'Chrysophyllum_pomiferum', 'Jacaranda_copaia', 'Pouteria_guianensis', 'Bagassa_guianensis', 'Eperua_falcata', 'Brosimum_rubescens', 'Humiriastrum_subcrenatum', 'Chrysophyllum_prieurii', 'Couratari_multiflora', 'Tapirira_guianensis', 'Eschweilera_coriacea', 'Eschweilera_sagotiana', 'Pouteria_torta', 'Helicostylis_tomentosa', 'Eriotheca_longitubulosa', 'Recordoxylon_speciosum', 'Pouteria_eugeniifolia', 'Licaria_chrysophylla', 'Micropholis_melinoniana', 'Goupia_glabra', 'Cary

In [ ]:
for k in range(len(UniqueList)):
  if UniqueList[k] in UniqueList1:
    continue
  else:  
    UniqueList1.append(UniqueList[k])
print(UniqueList1)

['Licania_membranacea', 'Parinari_montana', 'Pradosia_cochlearia', 'Lueheopsis_rugosa', 'Parkia_pendula', 'Tapura_capitulifera', 'Bocoa_prouacensis', 'Eperua_grandiflora', 'Platymiscium_pinnatum', 'Tabebuia_insignis', 'Carapa_surinamensis', 'Lecythis_poiteaui', 'Tachigali_melinonii', 'Parkia_nitida', 'Vouacapoua_americana', 'Licania_ovalifolia', 'Pouteria_ambelaniifolia', 'Dicorynia_guianensis', 'Protium_opacum', 'Pourouma_guianensis', 'Sextonia_rubra', 'Licaria_martiniana', 'Ormosia_coutinhoi', 'Licania_micrantha', 'Platonia_insignis', 'Chrysophyllum_pomiferum', 'Virola_michelii', 'Albizia_pedicellaris', 'Bagassa_guianensis', 'Eperua_falcata', 'Chrysophyllum_sanguinolentum', 'Brosimum_rubescens', 'Humiriastrum_subcrenatum', 'Hymenolobium_flavum', 'Chrysophyllum_prieurii', 'Inga_pezizifera', 'Couratari_multiflora', 'Tapirira_guianensis', 'Pourouma_melinonii', 'Parinari_campestris', 'Cecropia_obtusa', 'Eschweilera_coriacea', 'Eschweilera_sagotiana', 'Chaetocarpus_schomburgkianus', 'Inga